## cross-LLM model comparisons


In [1]:
llm_models = [
    "qwen2.5", "qwen2.5:14b", "llama3.1", "mistral", "nemotron-mini",
    "phi3.5", "gemma2"
]

In [2]:
from time import time
from datetime import datetime
import re
from pathlib import Path
import pandas as pd 
import ollama

from utils import *

In [3]:
pattern = r'```json\s*(.*?)\s*```'
file_log = "cross-llm-compare.log"
zi =  "人" # "person" # "字"  # "心"

lang = "CHN"
try:
    prompt_zi = LLM_PROMPT_ZI_DICT[lang]
except Exception as e:
    raise Exception(str(e))

Path(f"./llm_models/{lang}").mkdir(parents=True, exist_ok=True)

In [4]:
for model_name in llm_models:
    fp = Path(f"./llm_models/{lang}") / f"{lang}-{zi}-{model_name}-1.txt"    
    if fp.exists():
        log_msg(f"SKIP {zi} - {model_name} : Already processed - {fp} ", file_log)
        continue

    print(f"Calling '{model_name}' on '{zi}' ...")
    
    ts_now = datetime.now()
    ts_1 = time()
    try:
        resp = ollama.generate(model=model_name, prompt=prompt_zi.format(zi=zi))
    except Exception as e:
        log_msg(f"[ERROR] {zi} - {model_name} : Failed to call ollama.generate()", file_log)
        continue
        
    ts_2 = time()
    del_t = ts_2 - ts_1
    del_t_str = f"{del_t:.3f}"
    json_raw = resp.get('response', "")
    if not json_raw: 
        log_msg(f"[ERROR] No LLM response: {zi} - {model_name}", file_log)
        continue

    log_msg(f"[{str(ts_now)}] Completed {lang} - {zi} - {model_name} in {del_t_str} sec")
    with open(fp, "w", encoding="utf-8") as f:
        f.write(json_raw)

    
    # match = re.search(pattern, json_raw, re.DOTALL)

    # if match:
    #     json_string = match.group(1)
    #     if not json_string:
    #         log_msg(f"[WARN] No JSON extracted: {zi}")
    #         continue
            
    #     with open(fp, "w", encoding="utf-8") as f:
    #         f.write(json_string)
    # else:
    #     log_msg(f"[ERROR] No JSON extracted: {zi}", file_log)

Calling 'qwen2.5' on '人' ...
[2024-09-21 13:17:42.966245] Completed CHN - 人 - qwen2.5 in 85.095 sec
Calling 'llama3.1' on '人' ...
[2024-09-21 13:19:08.062054] Completed CHN - 人 - llama3.1 in 273.186 sec
Calling 'mistral' on '人' ...
[2024-09-21 13:23:41.248073] Completed CHN - 人 - mistral in 181.173 sec
Calling 'nemotron-mini' on '人' ...
[2024-09-21 13:26:42.421979] Completed CHN - 人 - nemotron-mini in 14.430 sec
Calling 'phi3.5' on '人' ...
[2024-09-21 13:26:56.853070] Completed CHN - 人 - phi3.5 in 128.467 sec
Calling 'gemma2' on '人' ...
[2024-09-21 13:29:05.321584] Completed CHN - 人 - gemma2 in 250.540 sec
